In [1]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.playlistItems.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/guides/code_samples#python

import os, sys
print( sys.version )

from time import sleep

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

from IPython.display import clear_output

from utils import Heartbeat

scopes = [
    "https://www.googleapis.com/auth/youtubepartner",
    "https://www.googleapis.com/auth/youtube",
    "https://www.googleapis.com/auth/youtube.force-ssl",
]
reqFreq = 1.0

3.9.13 (main, May 23 2022, 22:01:06) 
[GCC 9.4.0]


In [2]:
# Setup
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
api_service_name    = "youtube"
api_version         = "v3"
client_secrets_file = "../keys/client_secrets.json"

def get_flow_credentials_yt( client_secrets_file, scopes ):
    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, 
        scopes
    )
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, 
        api_version, 
        credentials = credentials
    )
    clear_output( wait = False ) # Prevent keys from being stored in output
    return flow, credentials, youtube

flow, credentials, youtube = get_flow_credentials_yt( client_secrets_file, scopes )

In [3]:
from pprint import pprint
from random import shuffle, choice
from urllib.error import HTTPError

def fetch_entire_playlist( playlistId, part ):
    """ Send serial requests until info about all playlist items is obtained """
    
    limiter = Heartbeat( reqFreq )
    
    # Init
    request = youtube.playlistItems().list(
        playlistId = playlistId ,
        part       = part ,
        maxResults = 50
    )
    response = request.execute()
    nextPage = response["nextPageToken"]
    items    = response['items']
    last     = False
    
    while 1:
        request = youtube.playlistItems().list(
            playlistId = playlistId,
            part       = part,
            pageToken  = nextPage,
            maxResults = 50
        )
        sleep( 1.0 )
        response = request.execute()
        try:
            nextPage = response["nextPageToken"]
        except KeyError:
            last = True
        items.extend( response['items'] )
        if last:
            break
        else:
            limiter.rest()
        
    return items


def reorder_entire_playlist( itemList, Nshort = 50 ):
    """ Given a list of video dictionaries, reorder them randomly in their parent playlist """
    N    = len( itemList )
    n    = int( N/2 ) 
    # posn = list( range( N-1, -1, -1 ) )
    posn = list( range( n , N ) )
    sccs = 0
    
    halfList = itemList[:n]
    shuffle( halfList )
    
    limiter = Heartbeat( reqFreq )
    
    print( f'\n########## PHASE 1 ##########\n' )
    
    for i, item in enumerate( halfList ):
        # Notify
        print( f'\n=== Item {i+1} of {n} ===\n' )
        # Reorder
        print( f'\tMoving {item["snippet"]["position"]} --to-> {posn[i]} ' )
        request = youtube.playlistItems().update(
            part = 'id,snippet',
            body = {
                'id': item['id'],
                'snippet': {
                    'playlistId': item['snippet']['playlistId'],
                    'resourceId': item['snippet']['resourceId'],
                    'position'  : posn[i],
                }
            }
        )
        sleep( 1.0 )
        try:
            response = request.execute()
        except HTTPError as err:
            if 0:
                sleep( 1.0 )
                response = request.execute()
            elif 0:
                print( "\tQUOTA MET, EXIT\n\t", err )
                break
            else:
                print( "\tERROR:\n\t", err )
                continue
        # Check
        if response['snippet']['position'] == posn[i]:
            sccs += 1
            print( f'\tSUCCESS: {response["snippet"]["position"]}' )
        else:
            print( f'\tFAILURE: {response["snippet"]["position"]}, desired {posn[i]}' )
        limiter.rest()
        
    
    print( f'\n########## PHASE 2 ##########\n' )
    
    idxs = list( range( N ) )
    shuffle( idxs )
    
    for i in range( Nshort ):
        ndxS = idxs.pop() # ----- Source index
        ndxD = idxs.pop() # ----- Destination index
        item = itemList[ ndxS ] # Source item
        
        # Notify
        print( f'\n=== Item {i+1} of {Nshort} ===\n' )
        # Reorder
        print( f'\tMoving {item["snippet"]["position"]} --to-> {ndxD} ' )
        request = youtube.playlistItems().update(
            part = 'id,snippet',
            body = {
                'id': item['id'],
                'snippet': {
                    'playlistId': item['snippet']['playlistId'],
                    'resourceId': item['snippet']['resourceId'],
                    'position'  : ndxD,
                }
            }
        )
        sleep( 1.0 )
        try:
            response = request.execute()
        except HTTPError as err:
            print( "\tERROR:\n\t", err )
            continue
        # Check
        if response['snippet']['position'] == ndxD:
            sccs += 1
            print( f'\tSUCCESS: {response["snippet"]["position"]}' )
        else:
            print( f'\tFAILURE: {response["snippet"]["position"]}, desired {ndxD}' )
        limiter.rest()
        
        
        
    # Notify
    print( f'Success: {sccs} / {n+Nshort}' )
    
    

In [4]:
ANALYZE = 0
REORDER = 1

for playlistId in [
        "PLxgoClQQBFjg29FkEQ_7rfDimHFzfFe9i", # Study Music 1
        "PLxgoClQQBFjilUnuo7hZNlulnTdn1f4DK", # Study Music 2
        "PLxgoClQQBFjjfRkJ-4zhVRn37rmczvTFx", # Study Music 3
        "PLxgoClQQBFjgiqSn5WkVa1slbCxTU12SI", # Study Music 4
        "PLxgoClQQBFji4MvFovpVY4MJJkaxUgiyi", # Study Music 5
        "PLxgoClQQBFjgxfyQm22P3MnbWyV74q4MR", # Study Music 6
        "PLxgoClQQBFjgUxgN0F02sPo4G22elRnz5", # Study Music 7
        "PLxgoClQQBFjj2lSERsDPqrnFDGEqQ6S2I", # Study Music 8
        "PLxgoClQQBFjiLGN3dkY6ALoySIcdMvHWU", # Study Music 9
        "PLxgoClQQBFjgTMrhvedWk8Q_CVLWwy3ak", # Pop Jams 1
        "PLxgoClQQBFjj6XdN2PlOsu5iiFaFLcoUu", # Pop Jams 2
        "PLxgoClQQBFjj9HB8q0IUZ8vP6yg8kQAzW", # Pop Jams 3
        "PLxgoClQQBFjjOg0gBqGUuhLwFVbsZA0Wj", # Pop Jams 4
    ][7:]:
    
    part       = "contentDetails,id,snippet,status" 
    items      = fetch_entire_playlist( playlistId, part )

    if ANALYZE:
        print( '\n\n==== Analyze! ====\n' )
        N      = len( items ) - 5
        repeat = 0
        unqSet = set([])

        for res in items:
            _id = res['id']
            if _id in unqSet:
                print( f"REPEAT found at {res['id']}" )
                repeat += 1
            else:
                unqSet.add( _id )

        print( f"{len(unqSet)} / {N} items are unique!" )
    
    if REORDER:
        print( '\n\n==== Reorder! ====\n' )
        reorder_entire_playlist( items )
        
        
    clear_output( wait = False )
        



==== Reorder! ====


########## PHASE 1 ##########


=== Item 1 of 150 ===

	Moving 110 --to-> 150 
	SUCCESS: 150

=== Item 2 of 150 ===

	Moving 13 --to-> 151 
	SUCCESS: 151

=== Item 3 of 150 ===

	Moving 17 --to-> 152 
	SUCCESS: 152

=== Item 4 of 150 ===

	Moving 31 --to-> 153 
	SUCCESS: 153

=== Item 5 of 150 ===

	Moving 28 --to-> 154 
	SUCCESS: 154

=== Item 6 of 150 ===

	Moving 73 --to-> 155 
	SUCCESS: 155

=== Item 7 of 150 ===

	Moving 90 --to-> 156 
	SUCCESS: 156

=== Item 8 of 150 ===

	Moving 123 --to-> 157 
	SUCCESS: 157

=== Item 9 of 150 ===

	Moving 61 --to-> 158 
	SUCCESS: 158

=== Item 10 of 150 ===

	Moving 25 --to-> 159 
	SUCCESS: 159

=== Item 11 of 150 ===

	Moving 60 --to-> 160 
	SUCCESS: 160

=== Item 12 of 150 ===

	Moving 139 --to-> 161 
	SUCCESS: 161

=== Item 13 of 150 ===

	Moving 34 --to-> 162 
	SUCCESS: 162

=== Item 14 of 150 ===

	Moving 53 --to-> 163 
	SUCCESS: 163

=== Item 15 of 150 ===

	Moving 70 --to-> 164 
	SUCCESS: 164

=== Item 16 of 150 ==

HttpError: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/playlistItems?part=id%2Csnippet&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">